In [2]:
!pip install torch_geometric
!pip install torch
!pip install torch_sparse
!pip install torch_scatter

  Using cached torch_geometric-2.2.0-py3-none-any.whl
  Using cached scikit_learn-1.2.1-cp310-cp310-macosx_12_0_arm64.whl (8.4 MB)
  Using cached psutil-5.9.4-cp38-abi3-macosx_11_0_arm64.whl (244 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.2-cp310-cp310-macosx_10_9_universal2.whl (17 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached torch-1.13.1-cp310-none-macosx_11_0_arm64.whl (53.2 MB)
  Using cached torch_sparse-0.6.16-cp310-cp310-macosx_11_0_arm64.whl
  Using cached torch_scatter-2.1.0.tar.gz (106 kB)
  Preparing metadata (setup.py) ... done
anceled
ERROR: Operation cancelled by user


In [2]:
from torch_geometric.nn.models.lightgcn import LightGCN
import pandas as pd
import os
from tqdm import tqdm
import torch

## Load Data
We can begin by loading in the user review data. For each user, we have a subset of the movies that they reviewed. We'll load each of the CSVs as dataframes, and store a dict of user IDs corresponding to their dataframes.

In [3]:
# for now we will use the first 10k rows of the data, set to None to use all data
AMOUNT_TO_LOAD = 100
EMBEDDING_DIM = 64

In [4]:
user_reviews_dir = 'user_reviews'
user_review_data = dict()

for filename in tqdm(os.listdir(user_reviews_dir)):
    if AMOUNT_TO_LOAD is not None and len(user_review_data) >= AMOUNT_TO_LOAD:
        break
    try:
        user_review_data[filename] = pd.read_csv(os.path.join(user_reviews_dir, filename), encoding='unicode_escape')
    except pd.errors.EmptyDataError:
        print(f'Empty file: {filename}')
        pass

  0%|          | 100/63111 [00:00<01:17, 814.07it/s]


Now let's split the data into training, validation, and test sets. Since this is a recommender, we're gonna split by removing some of the user's reviews.

For every user, so long as the user has more than 5 reviews, remove one review for the validation set and one review for the test set.

In [5]:
print(list(user_review_data.keys())[0])

asel82_reviews.csv


In [6]:
train_reviews = []
validation_reviews = []
test_reviews = []
for user_id, reviews in tqdm(user_review_data.items()):
    if len(reviews) > 5:
        # randomly remove one review from the user's reviews for the test set and one for the validation set
        reviews_to_remove = reviews.sample(2)
        # test data
        test_review_data = reviews_to_remove.iloc[0].to_dict()
        test_review_data['user_id'] = user_id
        test_reviews.append(test_review_data)
        # validation data
        validation_review_data = reviews_to_remove.iloc[1].to_dict()
        validation_review_data['user_id'] = user_id
        validation_reviews.append(validation_review_data)
        # train data
        train_review_data = reviews.drop(reviews_to_remove.index).to_dict('records')
        for review in train_review_data:
            review['user_id'] = user_id
        train_reviews.extend(train_review_data)
    else:
        # if the user has less than 5 reviews, we will use all of them for training
        train_review_data = reviews.to_dict('records')
        for review in train_review_data:
            review['user_id'] = user_id
        train_reviews.extend(train_review_data)

print(f'Train reviews: {len(train_reviews)}')
print(f'Validation reviews: {len(validation_reviews)}')
print(f'Test reviews: {len(test_reviews)}')

100%|██████████| 100/100 [00:00<00:00, 822.98it/s]

Train reviews: 45290
Validation reviews: 96
Test reviews: 96


In [7]:
train_reviews[0]

{'movie_title': 'All Too Well: The Short Film',
 'movie_rating': 4.0,
 'movie_id': 807762,
 'film_slug': '/film/all-too-well-the-short-film/',
 'user_id': 'asel82_reviews.csv'}

## Build the Model
Now that we have the training data, let's construct the model to train.

In [8]:
num_train_users = len(set([review['user_id'] for review in train_reviews]))
num_train_items = len(set([review['movie_id'] for review in train_reviews]))
num_nodes = num_train_users + num_train_items
print(f'Number of train users: {num_train_users}')
print(f'Number of train items: {num_train_items}')
print(f'Number of nodes: {num_nodes}')

Number of train users: 100
Number of train items: 12624
Number of nodes: 12724


In [9]:
# Let's map users to ids
user_to_id = dict()
for i, user_id in enumerate(set([review['user_id'] for review in train_reviews])):
    user_to_id[user_id] = i

# Let's map movies to ids
movie_to_id = dict()
for i, movie_id in enumerate(set([review['movie_id'] for review in train_reviews])):
    movie_to_id[movie_id] = i + num_train_users

In [10]:
# Let's remove any data in our validation and test sets that have ids that are not in our training set
# Before removal:
print(f'Validation reviews: {len(validation_reviews)}')
print(f'Test reviews: {len(test_reviews)}')

# Removal
validation_reviews = [review for review in validation_reviews if review['user_id'] in user_to_id and review['movie_id'] in movie_to_id]
test_reviews = [review for review in test_reviews if review['user_id'] in user_to_id and review['movie_id'] in movie_to_id]

# After removal:
print(f'Validation reviews: {len(validation_reviews)}')
print(f'Test reviews: {len(test_reviews)}')

Validation reviews: 96
Test reviews: 96
Validation reviews: 90
Test reviews: 87


In [11]:
import random

def convert_review_to_edge(review):
    user_id = user_to_id[review['user_id']]
    movie_id = movie_to_id[review['movie_id']]
    edge_weight = review['movie_rating']
    if (edge_weight < 3.5 and edge_weight > 2.5):
        return None, None
    edge = (user_id, movie_id)
    edge_weight = review['movie_rating']
    return edge, edge_weight

def shuffle_edges_and_edge_weights(edges, edge_weights):
    c = list(zip(edges, edge_weights))
    random.shuffle(c)
    return zip(*c)

def convert_reviews_to_edges(reviews):
    edges = []
    edge_weights = []
    for review in tqdm(reviews):
        edge, edge_weight = convert_review_to_edge(review)
        if edge is not None:
            edges.append(edge)
            edge_weights.append(edge_weight)
    
    # Reformat the edges to be a tensor
    edges = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edges, edge_weights

In [12]:
# Now let's create the edges between users and movies.
# The id of the user will be the index of the user in the user_to_id dict
# The id of the movie will be the index of the movie in the movie_to_id dict + the number of users

train_edges, train_edge_weights = convert_reviews_to_edges(train_reviews)
validation_edges, validation_edge_weights = convert_reviews_to_edges(validation_reviews)

print(f'Train edges: {train_edges.shape[1]}')
print(f'Validation edges: {validation_edges.shape[1]}')

100%|██████████| 90/90 [00:00<00:00, 720395.73it/s]

Train edges: 38672
Validation edges: 82


In [13]:
import torch_geometric.data as data

# create the graph
train_graph = data.Data(
    edge_index=train_edges,
    edge_attr=torch.tensor(train_edge_weights),
    num_nodes=num_nodes
)

validation_graph = data.Data(
    edge_index=validation_edges,
    edge_attr=torch.tensor(validation_edge_weights),
    num_nodes=num_nodes
)

In [14]:
train_graph.validate(raise_on_error=True)
validation_graph.validate(raise_on_error=True)

True

In [15]:
def resample_edges(positive_edges, negative_edges):
    """If the positive edges and negative edges are not the same length, resample the one that has more edges"""
    if positive_edges.shape[1] > negative_edges.shape[1]:
        positive_edges = positive_edges[:, torch.randperm(positive_edges.shape[1])[:negative_edges.shape[1]]]
    elif negative_edges.shape[1] > positive_edges.shape[1]:
        negative_edges = negative_edges[:, torch.randperm(negative_edges.shape[1])[:positive_edges.shape[1]]]
    return positive_edges, negative_edges

In [21]:
def compute_precision_at_k(model, num_items, num_users, positive_edges, k=5):
    """Compute the precision at k for the model, we have to proceed in batches"""
    model.eval()
    all_edges = torch.tensor([(user_id, item_id) for user_id in range(num_users) for item_id in range(num_items)], dtype=torch.long).t().contiguous()
    # create batches of edges
    all_edges = all_edges.split(10000, dim=1)
    with torch.no_grad():
        all_scores = []
        for batch in tqdm(all_edges):
            batch_scores = model(batch)
            all_scores.append(batch_scores)
        scores = torch.cat(all_scores, dim=0)
        # Get the top k items for each user
        top_k_scores, top_k_items = torch.topk(scores, k=k, dim=1)
        # Check how many of the top k items are in the positive edges
        num_correct = 0
        for user_id, positive_items in enumerate(positive_edges):
            num_correct += len(set(top_k_items[user_id].tolist()) & set(positive_items.tolist()))
        precision_at_k = num_correct / (num_users * k)
    return precision_at_k

def compute_precision_at_k_memory_efficient(model, num_items, num_users, positive_edges, k=5):
    model.eval()
    # select a random subset of 1000 users
    users = torch.randperm(num_users)[:1000]
    # for each user, use a heapq to keep track of the top k items
    top_k_items = [list() for _ in range(1000)]
    print("Created top k items")
    with torch.no_grad():
        # we're going to go over all possible item, user pairs, but we're going to do it in batches
        for user_id in tqdm(users):
            all_edges_for_user = torch.tensor([(user_id, item_id) for item_id in range(num_items)], dtype=torch.long).t().contiguous()
            top_k_items[user_id] = model(all_edges_for_user).topk(k=k, dim=0)[1].tolist()
        # Check how many of the top k items are in the positive edges
        num_correct = 0
        for user_id, positive_items in enumerate(positive_edges):
            # increment num_correct if the edge (user_id, item_id) is in the positive edges
            num_correct += len(set(top_k_items[user_id]) & set(positive_items.tolist()))
        precision_at_k = num_correct / (num_users * k)
    return precision_at_k

    

In [22]:
# Let's put this on tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [24]:
model = LightGCN(num_nodes=num_nodes, embedding_dim=EMBEDDING_DIM, num_layers=100)
optim = torch.optim.Adam(model.parameters(), lr=0.01)


train_positive_edges = train_graph.edge_index[:, train_graph.edge_attr >= 3.5]
train_negative_edges = train_graph.edge_index[:, train_graph.edge_attr <= 2.5]

BATCH_SIZE = min(train_positive_edges.shape[1], train_negative_edges.shape[1])

for epoch in range(1000):
    model.train()
    train_positive_edges, train_negative_edges = resample_edges(train_positive_edges, train_negative_edges)
    for i in tqdm(range(0, train_positive_edges.shape[1], BATCH_SIZE)):
        positive_edges = train_positive_edges[:, i:i+BATCH_SIZE]
        negative_edges = train_negative_edges[:, i:i+BATCH_SIZE]
        positive_ranks = model(positive_edges)
        negative_ranks = model(negative_edges)
        train_loss = model.recommendation_loss(positive_ranks, negative_ranks)
        writer.add_scalar('Loss/train', train_loss, epoch)
        optim.zero_grad()
        train_loss.backward()
        optim.step()

    # model.eval()
    # validation_positive_edges = validation_graph.edge_index[:, validation_graph.edge_attr >= 3.5]
    # validation_negative_edges = validation_graph.edge_index[:, validation_graph.edge_attr <= 2.5]
    # validation_positive_edges, validation_negative_edges = resample_edges(validation_positive_edges, validation_negative_edges)
    # validation_positive_ranks = model(validation_positive_edges)
    # validation_negative_ranks = model(validation_negative_edges)
    # validation_loss = model.recommendation_loss(validation_positive_ranks, validation_negative_ranks)
    # print(f'Epoch: {epoch}, Validation Loss: {validation_loss}')
    # writer.add_scalar('Loss/validation', validation_loss, epoch)

    if (epoch % 100 == 0):
        precision_at_k = compute_precision_at_k_memory_efficient(model, num_train_items, num_train_users, train_positive_edges)
        print(f'Epoch: {epoch}, Precision at k: {precision_at_k}')
        writer.add_scalar('Precision at k', precision_at_k, epoch)


100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


Created top k items


100%|██████████| 100/100 [00:19<00:00,  5.06it/s]


Epoch: 0, Precision at k: 0.006


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


...

Created top k items


100%|██████████| 100/100 [00:23<00:00,  4.35it/s]


Epoch: 600, Precision at k: 0.006


  0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 